In [23]:
import numpy as np
import pandas as pd
import math

In [24]:
hour_data = np.load("data/hour_data.npy")

In [25]:
hour_pm25 =hour_data[...,0]

In [26]:
station_path = "/home/yuxuan/wqy/Air_inference/data/"
stations = pd.read_csv("/home/yuxuan/wqy/Air_inference/data/" + '1085_stations.csv')

In [56]:
stations['pm25']=hour_pm25

In [65]:
# 定义左上角和右下角的经纬度坐标
top_left = (73.683851, 53.714166)
bottom_right = (135.383069, 18.424216)

# 计算经度和纬度的跨度
longitude_span = bottom_right[0] - top_left[0]
latitude_span = top_left[1] - bottom_right[1]


# 根据边长计算每个网格在经度和纬度上的跨度
grid_length = 1.0 # 1km
grid_longitude_span = grid_length / (111.12 * math.cos(math.radians(top_left[1])))
grid_latitude_span = grid_length / 111.12

# 计算经度和纬度上的网格数
num_longitude_grids = int(np.ceil(longitude_span / grid_longitude_span))
num_latitude_grids = int(np.ceil(latitude_span / grid_latitude_span))

# 生成网格的经纬度坐标和初始值为0的字典
grids = np.zeros((num_latitude_grids,num_longitude_grids))


# 假设DataFrame变量名为df

# 遍历DataFrame中的每一行数据
for index, row in stations.iterrows():
    longitude = row['longitude']
    latitude = row['latitude']
    pm25 = row['pm25']
    
    if top_left[0] <= longitude <= bottom_right[0] and bottom_right[1] <= latitude <= top_left[1]:
        # 计算相对于左上角的经纬度
        relative_longitude = longitude - top_left[0]
        relative_latitude = top_left[1] - latitude
        
        # 计算位于哪个网格
        grid_longitude = int(np.floor(relative_longitude / grid_longitude_span))
        grid_latitude = int(np.floor(relative_latitude / grid_latitude_span))
        
        grids[grid_latitude][grid_longitude] = pm25



In [61]:
grids

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [40]:
def fill_missing_values(grid):
    """
    使用局部插值法填充网格中的空值
    """
    # 找到不为0的点
    nonzero_points = np.argwhere(grid != 0)
    
    # 对每个不为0的点周围的网格进行插值
    for point in nonzero_points:
        x, y = point
        for i in range(x-1, x+2):
            for j in range(y-1, y+2):
                if i >= 0 and i < grid.shape[0] and j >= 0 and j < grid.shape[1] and grid[i][j] == 0:
                    # 对空值进行插值
                    grid[i][j] = np.mean(grid[max(0, i-1):min(i+2, grid.shape[0]), max(0, j-1):min(j+2, grid.shape[1])])
    
    return grid

In [17]:
nonzero_points = np.argwhere(grids != 0)

In [52]:
grid_1km = grids

In [66]:
import numpy as np
from scipy.interpolate import griddata
from scipy.interpolate import Rbf
# 将已有值的网格点和对应的值转换为数组
points = []
values = []
for lat in range(num_latitude_grids):
    for lon in range(num_longitude_grids):
        if grids[lat, lon] != 0:
            points.append([lat, lon])
            values.append(grids[lat, lon])

points = np.array(points)
values = np.array(values)

# 移除数组中的缺失值
mask = ~np.isnan(values)
points = points[mask]
values = values[mask]

# 生成新的网格点坐标
new_points = np.array(np.meshgrid(range(num_latitude_grids), range(num_longitude_grids))).T.reshape(-1, 2)

# 执行插值
rbf = Rbf(points[:, 0], points[:, 1], values, function='linear')
interpolated_values = rbf(new_points[:, 0], new_points[:, 1])

# 将插值结果填充到网格中
mask = grids == 0
grids[mask] = interpolated_values[:np.sum(mask)]